In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy

import math
import json
import io
import glob
import re

In [2]:
# load dictionary of country ids to country names, file created by country_id_dictionary.ipynb
# translate r_id and p_id to country names
i2c_json = open('id_to_country.json').read()
string_io = io.StringIO(i2c_json)
i2c = json.load(string_io)

In [3]:
# parameters
YEAR = 2009 # default year if not specified when calling functions
TTD = 'C_2' # C_2 - exports, C_1 - imports. TTD stands for trade type and direction. Type is always commodity

# # Graph building functions # #

In [4]:
# build edgelist for trade in particular direction (export or import)
def build_edgelist(year=YEAR, ttd=TTD):
    edge_list = []
    r_id_list = []
    reporters = glob.glob(f'./{year}_wt/{ttd}/*.txt')
    r_id_regex = re.compile(f'./{year}_wt/{ttd}\\\\([0-9]*).txt')
    for r_name in reporters:
        r_id = re.match(r_id_regex, r_name).group(1)
        r = pd.read_csv(r_name)
        if r.columns.size > 1:
            r_id_list.append(r_id)
        else:
            continue
        trade_good_ids = r[r.columns[0]].tolist() # first column are trade good ids
        for col in r.columns[1:]:
            # remove second column if don't want total exports of particular product to all countries
            # each column is a partner region
            trade_values_dict = {}
            i = 0
            for row in r[col]:
                if not math.isnan(row):
                    trade_good_id = trade_good_ids[i]
                    trade_values_dict[str(trade_good_id)] = row
                i += 1

            edge = (r_id, col, trade_values_dict)
            edge_list.append(edge)
    return edge_list, r_id_list

In [5]:
# combined import export edgelist 
#     (fills in unreported edges, e.g. China doesn't report anything some years, 
#     we can reconstruct their trade by reports of other countries)
def build_graph(year):
    exp_wt, exp_reporter_ids = build_edgelist(year, 'C_2')
    imp_wt, imp_reporter_ids = build_edgelist(year, 'C_1')
    graph_wt = nx.DiGraph(exp_wt)
    reporting_discrepancies = []
    for imp_edge in imp_wt:
        p_id, r_id, trade_values_dict = imp_edge # r and p ids are inversed to find exp relation rather than imp

        if not graph_wt.has_edge(r_id, p_id):
            graph_wt.add_edge(r_id, p_id)
            graph_wt[r_id][p_id].update(trade_values_dict)
        else:
            r_trade_values = graph_wt[r_id][p_id]
            unequal_reports, reported_by_exporter, reported_by_importer = 0, 0, 0 # statistics

            for k, v in r_trade_values.items():
                # trade value defined as max of exporter reported and importer reported values
                if k in trade_values_dict:
                    max_val = max(trade_values_dict[k], v)
                    if trade_values_dict[k] != v: unequal_reports += 1 # statistics
                else:
                    # trade goods only reported by exporter
                    max_val = v
                    reported_by_exporter += 1 # statistics
                r_trade_values[k] = max_val
            for k in trade_values_dict.keys():
                # add trade goods only reported by importer
                if k not in r_trade_values:
                    v = trade_values_dict[k]
                    r_trade_values[k] = v
                    reported_by_importer += 1 # statistics

            discrepancies = {'exp': reported_by_exporter, 'imp': reported_by_importer, 'ineq': unequal_reports} # statistics
            reporting_discrepancies.append((r_id, p_id, discrepancies)) # statistics
    return graph_wt, exp_reporter_ids, imp_reporter_ids, reporting_discrepancies

In [6]:
def calc_total_imp_exp(graph):
# for graph, create a global source/sink node for trade (warning: this effectively doubles the total trade value in the graph)
    total_exports_per_exporter = []
    for node in graph.nodes():
        global_exports = {}
        for neighbor in graph[node].values():
            for k, v in neighbor.items():
                if not k in global_exports:
                    global_exports[k] = v
                else:
                    global_exports[k] += v
        total_exports_per_exporter.append((node, global_exports))
    total_edge_list = [(r_id, '0', val) for (r_id, val) in total_exports_per_exporter]
    graph.add_edges_from(total_edge_list)
    
    total_imports_per_importer = {}
    for node in graph.nodes():
        for neighbor_id in graph[node].keys():
            val = graph[node][neighbor_id]
            if not neighbor_id in total_imports_per_importer:
                total_imports_per_importer[neighbor_id] = {}
            for k, v in val.items():
                if not k in total_imports_per_importer[neighbor_id]:
                    total_imports_per_importer[neighbor_id][k] = v
                else:
                    total_imports_per_importer[neighbor_id][k] += v
    total_edge_list = [('0', p_id, val) for p_id, val in total_imports_per_importer.items()]
    graph.add_edges_from(total_edge_list)
    
def calc_total_trade_value(graph):
    # calculate total trade value over all trade goods between two nodes
    for node in graph.nodes():
        for neighbor_dict in graph[node].values():
            total_trade = sum(neighbor_dict.values())
            neighbor_dict['total'] = total_trade

In [7]:
def build_threshold_graph(graph, threshold, trade_good=None):
    # trade good: number from 01 to 99, 'total' means total
    th_edge_list = []
    for node in graph.nodes():
        total_exports_by_product = graph[node]['0'] # 0 here is global source/sink
        threshold_vals = {k: v*threshold for k, v in total_exports_by_product.items()}
        for neighbor_id, neighbor_val in graph[node].items():
            th_exports = {}
            if trade_good == None:
                for tg_id in neighbor_val:
                    val = neighbor_val[tg_id]
                    if val >= threshold_vals[tg_id]:
                        th_exports[tg_id] = val
            elif trade_good in neighbor_val.keys():
                val = neighbor_val[trade_good]
                if val >= threshold_vals[trade_good]:
                    th_exports[trade_good] = neighbor_val[trade_good]

            if th_exports: # if not empty
                th_edge_list.append((node, neighbor_id, th_exports))

    return nx.DiGraph(th_edge_list)

In [8]:
def build_services_graph(year):
    # NOT USED IN FINAL ANALYSIS
    raw = pd.read_csv(f'./{year}_wt/S_2.csv')
    extracted_cols = raw[[raw.columns[x] for x in [3, 6, 11, -1]]]
    graph_tree = {}
    for entry in extracted_cols.to_numpy():
        a, b, c, d = entry
        a, b = str(a), str(b)
        if not a in graph_tree:
            graph_tree[a] = {}
        if not b in graph_tree[a]:
            graph_tree[a][b] = {}
        if c in graph_tree[a][b]:
            print('DEBUG: duplicate unique keys')
        graph_tree[a][b][c] = d
    
    # build edge list
    edge_list = []
    for a, a_val in graph_tree.items():
        for b, b_val in a_val.items():
            edge = (a, b, b_val)
            edge_list.append(edge)
    return nx.DiGraph(edge_list)

# # Build graphs # #

In [9]:
# graphs for 2009

graph_2009_wt, exp_r, imp_r, rep_discrep = build_graph(2009)
graph_2009_wt.remove_node('0') # remove Comtrade "0" (total trade), this node only contains trade in one direction, 
# and does not consider unreported trade (i.e. if the reporter does not report any trade, the total value is 0)
# we calculate our own "0" node (global source and sink node)
graph_2009_wt_no_world = graph_2009_wt.copy()

# # add a global source and sink for exports
calc_total_imp_exp(graph_2009_wt)
calc_total_trade_value(graph_2009_wt)

# threshold graphs
th_graph_2009_wt = build_threshold_graph(graph_2009_wt, 0.01, trade_good = 'total') # number from 1 to 99, 'total' means total
th_graph_2009_wt_no_world = th_graph_2009_wt.copy()
th_graph_2009_wt_no_world.remove_node('0')

In [10]:
# graphs for 2019

graph_2019_wt, exp_r, imp_r, rep_discrep = build_graph(2019)
graph_2019_wt.remove_node('0')
graph_2019_wt_no_world = graph_2019_wt.copy()

# add a global source and sink for exports
calc_total_imp_exp(graph_2019_wt)
calc_total_trade_value(graph_2019_wt)

# threshold graphs
th_graph_2019_wt = build_threshold_graph(graph_2019_wt, 0.01, trade_good = 'total') # number from 1 to 99, 'total' means total
th_graph_2019_wt_no_world = th_graph_2019_wt.copy()
th_graph_2019_wt_no_world.remove_node('0')

# # Calculate Katz centralities for different years and thresholds # #

In [11]:
# calculate katz centrality
def katz(g):
    phi = max(nx.adjacency_spectrum(g))
    print(phi)
    return nx.algorithms.centrality.katz_centrality_numpy(g, abs(1/(phi+1)))

def build_graph_for_year_and_threshold(year, threshold):
    g, exp_r, imp_r, rep_discrep = build_graph(year)
    g_no_world = g.copy()
    g_no_world.remove_node('0')

    # add a global source and sink for exports CAN ONLY BE RUN ONCE
    calc_total_imp_exp(g)
    calc_total_trade_value(g)

    th_g = build_threshold_graph(g, threshold, trade_good = 'total') # number from 01 to 99, 'total' means total
    th_g.remove_node('0')
    th_g_no_world = th_g.copy()
    return th_g_no_world

def calc_all_katz_centralities(years, thresholds):
    katz_centralities_dict = {}
    for year in years:
        for th in thresholds:
            key = f'{year}_{th}'
            if key not in katz_centralities_dict:
                katz_centralities_dict[key] = {}
            g_for_katz = build_graph_for_year_and_threshold(year, th)
            for k, v in katz(g_for_katz).items():
                katz_centralities_dict[key][k] = v
    return katz_centralities_dict

In [12]:
katz_centralities_dict = calc_all_katz_centralities(years=[2009, 2019], thresholds=[0, 0.01, 0.02])

# get top 10 central countries for each year*threshold combination
top_10s = []
for key, kc in katz_centralities_dict.items():
    kcs = [k for k,v in sorted(kc.items(), key=lambda x:x[1], reverse=True)[:10]]
    top_10s.extend(kcs)

# get katz centralities for countries in top 10s
kcs_dict = {}
for key, kc in katz_centralities_dict.items():
    key_list = []
    for k, v in kc.items():
        if k in set(top_10s):
            if not i2c[k] in kcs_dict:
                kcs_dict[i2c[k]] = {}
            kcs_dict[i2c[k]][key] = v

pd.DataFrame(kcs_dict)
# .to_csv('katz_over_time.csv')


(152.3407829830127+0j)
(17.029698765600543+0j)
(9.118928896228622+0j)
(135.9871582017076+0j)
(17.53485516732878+0j)
(10.33170048365354+0j)


,France,Italy,Thailand,Turkey,Austria,Belgium,Czechia,"China, Hong Kong SAR",Netherlands,Poland,...,Japan,Rep. of Korea,USA,China,Ireland,South Africa,Indonesia,Malaysia,Mexico,India
2009_0,0.092374,0.089659,0.090188,0.088287,0.090760,0.090734,0.091396,0.085866,0.090879,0.091055,...,0.090248,0.090004,0.090438,0.090392,0.089771,0.090531,0.088804,0.090504,0.090409,0.089487
2009_0.01,0.256931,0.228996,0.119506,0.149370,0.094735,0.199217,0.082283,0.155825,0.258351,0.142185,...,0.244626,0.180513,0.314419,0.295281,0.052249,0.017879,0.046547,0.094196,0.080540,0.201272
2009_0.02,0.308529,0.218550,0.032334,0.064146,0.047145,0.244284,0.019774,0.177713,0.296578,0.048756,...,0.232131,0.170478,0.425059,0.396008,0.037653,0.009276,0.022825,0.024345,0.088786,0.112389
2019_0,0.055676,0.095911,0.097389,0.095143,0.095816,0.097007,0.097112,0.092483,0.055943,0.097062,...,0.095360,0.096606,0.055943,0.096186,0.096986,0.097663,0.096716,0.094219,0.093123,0.094812
2019_0.01,0.049451,0.210493,0.121084,0.197877,0.126506,0.217899,0.136528,0.178324,0.072565,0.172241,...,0.248877,0.223164,0.114360,0.333156,0.054758,0.052895,0.104893,0.119225,0.149667,0.221940
2019_0.02,0.025079,0.180093,0.150441,0.122001,0.062630,0.209212,0.050212,0.246720,0.035698,0.130274,...,0.305427,0.228797,0.120208,0.429864,0.034309,0.034175,0.127827,0.159159,0.183810,0.248345


# # Other statistics # #

In [13]:
# node degree centrality
centrality = nx.algorithms.centrality.out_degree_centrality(th_graph_2009_wt_no_world)
sorted_centrality = {k: v for k, v in sorted(centrality.items(), key=lambda item: item[1], reverse=True)}
sorted_centrality


{'32': 0.11618257261410789,
 '381': 0.11203319502074689,
 '410': 0.11203319502074689,
 '300': 0.1078838174273859,
 '643': 0.1078838174273859,
 '842': 0.1078838174273859,
 '818': 0.1078838174273859,
 '699': 0.1078838174273859,
 '748': 0.1078838174273859,
 '792': 0.1037344398340249,
 '470': 0.1037344398340249,
 '40': 0.09958506224066391,
 '804': 0.09958506224066391,
 '710': 0.09958506224066391,
 '688': 0.0954356846473029,
 '826': 0.0954356846473029,
 '760': 0.0954356846473029,
 '392': 0.0954356846473029,
 '76': 0.0954356846473029,
 '288': 0.0954356846473029,
 '686': 0.0954356846473029,
 '384': 0.0954356846473029,
 '251': 0.0912863070539419,
 '348': 0.0912863070539419,
 '704': 0.0912863070539419,
 '404': 0.0912863070539419,
 '800': 0.0912863070539419,
 '428': 0.0912863070539419,
 '554': 0.0912863070539419,
 '152': 0.0912863070539419,
 '454': 0.0912863070539419,
 '268': 0.08713692946058091,
 '642': 0.08713692946058091,
 '616': 0.08713692946058091,
 '156': 0.08713692946058091,
 '191': 0.087

In [14]:
def compute_graph_centrality(node_centralities):
    g = graph_2009_wt_no_world.number_of_nodes()
    max_centrality_in_graph = max(node_centralities, key=lambda x:x[1])[1]
    sum_diff_max_and_actual_centrality = max_centrality_in_graph * g - sum([x[1] for x in node_centralities])
    sum_diff_max_and_actual_centrality_starnet = (g-1)*(g-2)
    freeman_centralization_index = sum_diff_max_and_actual_centrality / sum_diff_max_and_actual_centrality_starnet
    return freeman_centralization_index

In [15]:
# compute individual node and graph degree centralities
g = graph_2009_wt_no_world.number_of_nodes()
export_node_centrality = [(k,v/(g-1)) for k,v in graph_2009_wt_no_world.out_degree]
import_node_centrality = [(k,v/(g-1)) for k,v in graph_2009_wt_no_world.in_degree]

compute_graph_centrality(import_node_centrality), compute_graph_centrality(export_node_centrality)

(0.0018165722582829732, 0.0017124073850886402)

In [16]:
# compute network edge density
L = graph_2009_wt_no_world.number_of_edges()
density = L/(g*(g-1))
density

0.5326291965296115

In [17]:
# compute directed graph clustering (triangles)
# modified from networkx source code
def triples_and_directed_triangles(G, export_direction=True, nodes=None):
    """Return an iterator of
    (node, total_connected_triples, directed_triangles).
    Used for directed clustering.
    This function counts directed triangles 
    so bidirectional links between j and k are counted separately.
    total_connected_triples indicates triples in which i is the middle node
    """
    if export_direction:
        nodes_nbrs = ((n, G._succ[n]) for n in G.nbunch_iter(nodes))
    else:
        nodes_nbrs = ((n, G._pred[n]) for n in G.nbunch_iter(nodes))

    for i, succs in nodes_nbrs:
        if i == '0': continue
        isuccs = set(succs) - {i}

        directed_triangles = 0
        for j in isuccs:
            jsuccs = set(G._succ[j]) - {j}
            directed_triangles += sum(
                1 for k in (isuccs & jsuccs)
            )
        dout = len(isuccs)
        ttotal = int((dout * (dout - 1)) / 2) # result always integer
        yield (i, ttotal, directed_triangles)

def calc_clustering_coefficient(G, export_direction=True):
    triples_and_triangles_per_node = triples_and_directed_triangles(G, export_direction=export_direction)
    total_triples = 0
    total_triangles = 0
    for node, triples, triangles in triples_and_triangles_per_node:
        total_triples += 2 * triples
        total_triangles += triangles
    
    return total_triangles / total_triples

In [18]:
# how likely are countries both receiving imports/exports from country A to export to each other? (any direction counts)
export_clustering = calc_clustering_coefficient(graph_2009_wt_no_world, export_direction=True)
import_clustering = calc_clustering_coefficient(graph_2009_wt_no_world, export_direction=False)
print(export_clustering, import_clustering)

0.7568221964717149 0.7248036767415035
